In [209]:
import pandas as pd
from datetime import datetime

In [210]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [211]:
cols=['Name','Market Cap (US$ Billion)']

In [212]:
def extract():
    df = pd.DataFrame(cols)
    df = extract_from_json("bank_market_cap.json")
    return df

In [213]:
# Write your code here
def extract_exchange_rates():
    exchange_rate = pd.read_csv("exchange_rates.csv")
    exchange_rate = exchange_rate.rename(columns={"Unnamed: 0":"Currency"})
    return exchange_rate

In [289]:
def transform(market_cap, currency):
    all_rates = extract_exchange_rates()
    rate = all_rates.loc[lambda all_rates: all_rates['Currency'] == currency]['Rates']
    market_cap["Market Cap (US$ Billion)"] = round(market_cap["Market Cap (US$ Billion)"].multiply(rate.iloc[0]), 3)
    market_cap = market_cap.rename(columns={"Market Cap (US$ Billion)":"Market Cap ({} Billion)".format(currency)})
    return market_cap

In [290]:
def load(bank_market_cap, currency):
    bank_market_cap.to_csv("bank_market_cap_{currency}.csv".format(currency), index=False)
    

In [291]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [310]:
def get_currency():
    user_input = input("Enter a currency that you would like to see (Default = USD):")
    all_rates = extract_exchange_rates()
    currency = "USD"
    for rate in all_rates['Currency']:
        if (user_input == rate):
            currency = user_input
    return currency


get_currency()

AUD
BGN
BRL
CAD
CHF
CNY
CZK
DKK
EUR
GBP
HKD
HRK
HUF
IDR
ILS
INR
ISK
JPY
KRW
MXN
MYR
NOK
NZD
PHP
PLN
RON
RUB
SEK
SGD
THB
TRY
USD
ZAR


In [292]:
# Write your code here
log("ETL Job Started")
log("Extract phase Started")

In [293]:
# Call the function here
extracted_data = extract()

# Print the rows here
extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [294]:
# Write your code here
log("Extract phase Ended")

In [295]:
# Write your code here
log("Transform phase Started")

In [304]:
# Call the function here
currency = get_currency()
transformed_data = transform(extracted_data, currency)
# Print the first 5 rows here
transformed_data.head()


e


IndexError: single positional indexer is out-of-bounds

In [230]:
# Write your code here
log("Transform phase Ended")

In [231]:
# Write your code here
log("Load phase Started")

In [ ]:
# Write your code here
load(transformed_data, currency)

In [163]:
# Write your code here
log("Load phase Ended")

In [272]:
currency = "GBP"
all_rates = extract_exchange_rates()
# rate = float(all_rates[all_rates['Currency'] == currency]['Rates'])
rate = all_rates.loc[lambda all_rates: all_rates['Currency'] == currency]['Rates']
print(float(rate.iloc[0]))

0.7323984208000001
